In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('stats.csv')

## Calculating Best Global Heroes

In [232]:
class best_Heroes:
    def __init__(self, data):
        self.data = data
        self.hero_stats = {}
        self._populate_hero_stats(mirrored = False)
        self.hero_categories = [
            {"name": "Ana", "category": ["Poke", "Brawl", "Dive"]},
            {"name": "Ashe", "category": ["Poke", "Brawl"]},
            {"name": "Bap", "category": ["Brawl", "Poke"]},
            {"name": "Bastion", "category": ["Poke", "Brawl"]},
            {"name": "Brig", "category": ["Dive"]},
            {"name": "Cass", "category": ["Brawl"]},
            {"name": "DVA", "category": ["Brawl", "Dive"]},
            {"name": "Doom", "category": ["Brawl", "Dive"]},
            {"name": "Echo", "category": ["Dive", "Brawl"]},
            {"name": "Genji", "category": ["Dive", "Brawl", "Poke"]},
            {"name": "Hanzo", "category": ["Poke"]},
            {"name": "Hazard", "category": ["Brawl"]},  
            {"name": "Illari", "category": ["Poke"]},  
            {"name": "Queen", "category": ["Brawl"]},
            {"name": "Junk", "category": ["Brawl"]},
            {"name": "Juno", "category": ["Dive", "Brawl"]},  
            {"name": "Kiri", "category": ["Dive", "Poke", "Brawl"]}, 
            {"name": "LW", "category": ["Dive"]},
            {"name": "Lucio", "category": ["Brawl", "Dive"]}, 
            {"name": "Mauga", "category": ["Brawl"]},
            {"name": "Mei", "category": ["Brawl", "Poke"]}, 
            {"name": "Mercy", "category": ["Poke"]},
            {"name": "Moira", "category": ["Dive", "Brawl"]},
            {"name": "Orisa", "category": ["Brawl", "Poke"]},
            {"name": "Pharah", "category": ["Poke"]},
            {"name": "Ram", "category": ["Brawl", "Poke"]},
            {"name": "Reaper", "category": ["Brawl", "Dive"]},
            {"name": "Rein", "category": ["Brawl"]},
            {"name": "Hog", "category": ["Poke"]},
            {"name": "Sigma", "category": ["Poke", "Brawl"]},
            {"name": "Soj", "category": ["Poke"]},  
            {"name": "Soldier", "category": ["Poke"]},
            {"name": "Sombra", "category": ["Dive"]},
            {"name": "Sym", "category": ["Brawl"]},
            {"name": "Torb", "category": ["Poke", "Brawl"]},  
            {"name": "Tracer", "category": ["Dive"]}, 
            {"name": "Venture", "category": ["Dive", "Brawl"]},  
            {"name": "Widow", "category": ["Poke"]},  
            {"name": "Winston", "category": ["Dive"]},
            {"name": "Ball", "category": ["Dive"]}, 
            {"name": "Zarya", "category": ["Brawl"]},  
            {"name": "Zen", "category": ["Poke"]}  
            ]
    def _populate_hero_stats(self, mirrored):
        # Iterate through the rows of the data to populate hero statistics
        for idx, row in self.data.iterrows():
            result = row['Result']
            allied_heroes = row['My_Tank'].split(',') + row['My_DPS'].split(',') + row['My_Sups'].split(',') 
            enemy_heroes = row['E_Tank'].split(',') + row['E_DPS'].split(',') + row['E_Sups'].split(',')


            all_heroes = set(allied_heroes + enemy_heroes)  # Set for checking duplicates
            
            for hero in all_heroes:
                if not mirrored and hero in allied_heroes and hero in enemy_heroes:
                    continue  

                if hero not in self.hero_stats:
                    if hero in row['My_Tank'].split(',') + row['E_Tank'].split(','):
                        role = 'Tank'
                    elif hero in row['My_DPS'].split(',') + row['E_DPS'].split(','):
                        role = 'DPS'
                    elif hero in row['My_Sups'].split(',') + row['E_Sups'].split(','):
                        role = 'Support'
                    else:
                        role = 'Unknown'

                    self.hero_stats[hero] = {'matches': 0, 'wins': 0, 'Role': role}

                if hero in allied_heroes:
                    self.hero_stats[hero]['matches'] += 1
                    if result == 1:
                        self.hero_stats[hero]['wins'] += 1
                if hero in enemy_heroes:
                    self.hero_stats[hero]['matches'] += 1
                    if result == 0:
                        self.hero_stats[hero]['wins'] += 1
                
        # remove heroes with no matches
        self.hero_stats = {hero: stats for hero, stats in self.hero_stats.items() if stats['matches'] > 0}

    def get_bayesian_winrate(self, match_wins, hero_param=False, a=25, b=25):
        bayesian_winrates: dict[str, list[int]] = {}
        if hero_param:
            # Define a mapping for hero-specific (a, b) values
            hero_bayesian_params = {
            "Ana": (27.5, 22.5),  # Default Bayesian values for Ana
            "Ashe": (27.5, 22.5),
            "Bap": (27.5, 22.5),
            "Bastion": (25, 25),
            "Brig": (32.5, 17.5),
            "Cass": (20, 30),
            "DVA": (25, 25),
            "Doom": (25, 25),
            "Echo": (27.5, 22.5),
            "Genji": (25, 25),
            "Hanzo": (25, 25),
            "Hazard": (25, 25),
            "Illari": (25, 25),
            "Queen": (27.5, 22.5),
            "Junk": (25, 25),
            "Juno": (20, 30),
            "Kiri": (30, 20),
            "LW": (25, 25),
            "Lucio": (25, 25),
            "Mauga": (25, 25),
            "Mei": (30, 20),
            "Mercy": (25, 25),
            "Moira": (25, 25),
            "Orisa": (25, 25),
            "Pharah": (25, 25),
            "Ram": (25, 25),
            "Reaper": (27.5, 22.5),
            "Rein": (25, 25),
            "Hog": (25, 25),
            "Sigma": (25, 25),
            "Soj": (25, 25),
            "Soldier": (20, 30),
            "Sombra": (25, 25),
            "Sym": (25, 25),
            "Torb": (25, 25),
            "Tracer": (30, 20),
            "Venture": (25, 25),
            "Widow": (12.5, 37.5),
            "Winston": (27.5, 22.5),
            "Ball": (25, 25),
            "Zarya": (25, 25),
            "Zen": (25, 25)
        }
        for hero in match_wins:
            if hero_param:
                a, b = hero_bayesian_params[hero]
            matches = match_wins[hero]['matches']
            wins = match_wins[hero]['wins']
            bayesian_winrates[hero] = (wins + a) / (matches + a + b)  # Posterior mean of winrate
        
        return bayesian_winrates
    


    def normalise_winrates(self, winrates):
        """Normalises the winrates across roles. 
        
        - Requires winrate
        
        
        """
        roles = {'Tank': [], 'DPS': [], 'Support': []}
        # Separate winrates by role
        for hero, winrate in winrates.items():
            role = self.hero_stats[hero]['Role']
            roles[role].append(winrate)
        
        normalised = {}
        
        # normalise winrates within each role
        for role, winrate_list in roles.items():
            winrate_array = np.array(winrate_list)
            min_winrate = np.min(winrate_array)
            max_winrate = np.max(winrate_array)
            
            for hero, winrate in winrates.items():
                if self.hero_stats[hero]['Role'] == role:
                    normalised[hero] = (winrate - min_winrate) / (max_winrate - min_winrate)
        
        return normalised

    def stat_visualiser(self, values, threshold = 0.9):
        # Separate heroes by role
        roles = {'Tank': [], 'DPS': [], 'Support': []}
        for hero, value in values.items():
            role = self.hero_stats[hero]['Role']
            if value >= threshold:
                roles[role].append((hero, value))
        
        # Sort each role by value in descending order
        for role in roles:
            roles[role].sort(key=lambda x: x[1], reverse=True)
        
        # Print header
        print(f"{'Tank':<20} {'DPS':<20} {'Support':<20}")
        print("-" * 60)
        
        # Print heroes side by side
        for i in range(max(len(roles['Tank']), len(roles['Support']), len(roles['DPS']))):
            tank = f"{roles['Tank'][i][0]}: {roles['Tank'][i][1]:.4f}" if i < len(roles['Tank']) else ""
            dps = f"{roles['DPS'][i][0]}: {roles['DPS'][i][1]:.4f}" if i < len(roles['DPS']) else ""
            support = f"{roles['Support'][i][0]}: {roles['Support'][i][1]:.4f}" if i < len(roles['Support']) else ""            
            print(f"{tank:<20} {dps:<20} {support:<20}")


    def first_pick(self, teammates, curr_map, comp = "Poke,Brawl,Dive"):
        """ Calculate the total matches and wins of all heroes that were played on a given map or with specific teammates"""

        comp_categories = set(comp.split(","))
        teammates = [hero.strip() for hero in teammates.split(',')] if isinstance(teammates, str) else teammates
        hero_stats: dict[str, list[int]] = {}

        for idx, row in self.data.iterrows():
            result = row['Result']
            allied_heroes = row['My_Tank'].split(',') + row['My_DPS'].split(',') + row['My_Sups'].split(',')
            enemy_heroes = row['E_Tank'].split(',') + row['E_DPS'].split(',') + row['E_Sups'].split(',')
            my_map = row['Map']

            # Check if any teammate is in allied heroes or if the map matches the current map
            self._synergy(teammates, curr_map, hero_stats, row, result, allied_heroes, enemy_heroes, my_map, comp_categories)


        for teammate in teammates:
            if teammate in hero_stats:
                del hero_stats[teammate]
        return hero_stats

    def _synergy(self, teammates, curr_map, hero_stats, row, result, allied_heroes, enemy_heroes, my_map, comp_categories):
        if any(teammate in allied_heroes for teammate in teammates) or my_map == curr_map:
            for hero in allied_heroes:
                
                hero_data = next((h for h in self.hero_categories if h["name"] == hero), None)
                if comp_categories.isdisjoint(hero_data['category']):
                    continue
                if hero not in hero_stats:
                    if hero in row['My_Tank'].split(',') + row['E_Tank'].split(','):
                        role = 'Tank'
                    elif hero in row['My_DPS'].split(',') + row['E_DPS'].split(','):
                        role = 'DPS'
                    elif hero in row['My_Sups'].split(',') + row['E_Sups'].split(','):
                        role = 'Support'
                    else:
                        role = 'Unknown'

                    hero_stats[hero] = {'matches': 0, 'wins': 0, 'Role': role}
                
                hero_stats[hero]['matches'] += 1
                if result == 1:
                    hero_stats[hero]['wins'] += 1

        if any(enemy in enemy_heroes for enemy in teammates) or my_map == curr_map:
            for hero in enemy_heroes:

                hero_data = next((h for h in self.hero_categories if h["name"] == hero), None)
                if comp_categories.isdisjoint(hero_data['category']):
                    continue

                if hero not in hero_stats:
                    if hero in row['My_Tank'].split(',') + row['E_Tank'].split(','):
                        role = 'Tank'
                    elif hero in row['My_DPS'].split(',') + row['E_DPS'].split(','):
                        role = 'DPS'
                    elif hero in row['My_Sups'].split(',') + row['E_Sups'].split(','):
                        role = 'Support'
                    else:
                        role = 'Unknown'

                    hero_stats[hero] = {'matches': 0, 'wins': 0, 'Role': role}
                hero_stats[hero]['matches'] += 1
                if result == 0:
                    hero_stats[hero]['wins'] += 1


    def matchup_comparison(self, a_matchup, e_matchup):
        a_matchup = {hero.strip() for hero in a_matchup.split(",")}
        e_matchup = {hero.strip() for hero in e_matchup.split(",")}

        results = {
            'Placeholder': {'matches': 0, 'wins': 0, 'Role': 'Unknown'}
        }

        for idx, row in self.data.iterrows():
            result = row['Result']
            allied_heroes = row['My_Tank'].split(',') + row['My_DPS'].split(',') + row['My_Sups'].split(',') 
            enemy_heroes = row['E_Tank'].split(',') + row['E_DPS'].split(',') + row['E_Sups'].split(',')

            all_heroes = set(allied_heroes + enemy_heroes)  # Set for checking duplicates

            if not((a_matchup.issubset(allied_heroes) and e_matchup.issubset(enemy_heroes)) or 
                   (a_matchup.issubset(enemy_heroes) and e_matchup.issubset(allied_heroes))):
                continue 

            results['Placeholder']['matches'] += 1   

            if (a_matchup.issubset(allied_heroes) and e_matchup.issubset(enemy_heroes) and result == 1) or \
               (a_matchup.issubset(enemy_heroes) and e_matchup.issubset(allied_heroes) and result == 0):
                results['Placeholder']['wins'] += 1

        return results


    def my_hero_performance(self):
        my_hero_perf: dict[str, list[int]] = {}
        for idx, row in self.data.iterrows():
            result = row['Result']
            my_hero = row['My_Hero']

            if my_hero in row['My_Tank'].split(','):
                my_role = 'Tank'
            elif my_hero in row['My_DPS'].split(','):
                my_role = 'DPS'
            elif my_hero in row['My_Sups'].split(','):
                my_role = 'Support'
            else:
                my_role = 'Unknown'

            if my_hero not in my_hero_perf:
                my_hero_perf[my_hero] = {'matches': 0, 'wins': 0, 'Role': my_role}

            my_hero_perf[my_hero]['matches'] += 1
            if result == 1:
                my_hero_perf[my_hero]['wins'] += 1

        return my_hero_perf
    

    def add_data(self, matches_1, matches_2):

        new_dataset = {}

        for hero_1 in matches_1:
            if hero_1 in matches_2:
                new_dataset[hero_1] = {'matches': matches_1[hero_1]['matches'] + matches_2[hero_1]['matches'], 
                                        'wins': matches_1[hero_1]['wins'] + matches_2[hero_1]['wins'], 'Role':matches_1[hero_1]['Role']}
        return new_dataset
            
            

    




df = df[df['Closeness'] == 0]  # Keep only rows where closeness is 0        
heroes = best_Heroes(df)
bayesian_performance = heroes.get_bayesian_winrate(heroes.hero_stats, True)
normalised = heroes.normalise_winrates(bayesian_performance)
heroes.stat_visualiser(normalised, 0)




# first_pick = heroes.first_pick("Soj, LW, Ana", "Hollywood")
# bayesian_winrates = heroes.get_bayesian_winrate(first_pick)
# normalised = heroes.normalise_winrates(bayesian_winrates)
# heroes.stat_visualiser(normalised, 0.75)

# matchups = heroes.matchup_comparison("Genji, Ana", "")





Tank                 DPS                  Support             
------------------------------------------------------------
Winston: 1.0000      Tracer: 1.0000       Kiri: 1.0000        
Ram: 0.8025          Mei: 0.9284          Brig: 0.9225        
Queen: 0.7778        Ashe: 0.8472         Moira: 0.7923       
Ball: 0.7701         Genji: 0.8295        Bap: 0.7399         
Mauga: 0.4575        Torb: 0.7937         Mercy: 0.6518       
Orisa: 0.4528        Sym: 0.7788          LW: 0.6326          
Hazard: 0.4485       Pharah: 0.7520       Ana: 0.6196         
Sigma: 0.2138        Reaper: 0.7489       Illari: 0.5654      
Hog: 0.2092          Echo: 0.7225         Lucio: 0.5335       
Zarya: 0.1222        Soj: 0.7166          Zen: 0.4402         
Rein: 0.0988         Venture: 0.6673      Juno: 0.0000        
Doom: 0.0897         Bastion: 0.6673                          
DVA: 0.0000          Sombra: 0.6424                           
                     Junk: 0.6424                        